In [1]:
import numpy as np
import pandas as pd
import csv

In [2]:
!mkdir data

In [14]:
!wget https://www.gutenberg.org/files/11/11-0.txt

--2018-09-10 21:41:34--  https://www.gutenberg.org/files/11/11-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 2610:28:3090:3000:0:bad:cafe:47, 152.19.134.47
Connecting to www.gutenberg.org (www.gutenberg.org)|2610:28:3090:3000:0:bad:cafe:47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173595 (170K) [text/plain]
Saving to: '11-0.txt'

11-0.txt            100%[===================>] 169.53K   128KB/s    in 1.3s    

2018-09-10 21:41:37 (128 KB/s) - '11-0.txt' saved [173595/173595]



In [15]:
!mv 11-0.txt data/wonderland.txt

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [20]:
filename = "data/wonderland.txt"
raw_text = open(filename, encoding='utf-8').read()
raw_text = raw_text.lower()

In [21]:
raw_text[:100]

'\ufeffproject gutenberg’s alice’s adventures in wonderland, by lewis carroll\n\nthis ebook is for the use o'

In [22]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [23]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163817
Total Vocab:  61


In [25]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

In [26]:
n_patterns = len(dataX)

In [27]:
print("Total Patterns: ", n_patterns)

Total Patterns:  163717


In [29]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [30]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [33]:
model.fit(X, y, epochs = 20, batch_size = 2048)

Epoch 1/20
163717/163717 [==============================] - 27s 165us/step - loss: 3.0623
Epoch 2/20
163717/163717 [==============================] - 27s 166us/step - loss: 3.0479
Epoch 3/20
163717/163717 [==============================] - 27s 166us/step - loss: 3.0006
Epoch 4/20
163717/163717 [==============================] - 27s 167us/step - loss: 2.9465
Epoch 5/20
163717/163717 [==============================] - 27s 167us/step - loss: 2.9214
Epoch 6/20
163717/163717 [==============================] - 27s 167us/step - loss: 2.9029
Epoch 7/20
163717/163717 [==============================] - 27s 167us/step - loss: 2.8847
Epoch 8/20
163717/163717 [==============================] - 27s 167us/step - loss: 2.8674
Epoch 9/20
163717/163717 [==============================] - 27s 167us/step - loss: 2.8520
Epoch 10/20
163717/163717 [==============================] - 27s 168us/step - loss: 2.8371
Epoch 11/20
163717/163717 [==============================] - 27s 168us/step - loss: 2.8215
Epoch 12

In [34]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [36]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print("\nDone.")

Seed:
" door of which was a bright brass
plate with the name ‘w. rabbit’ engraved upon it. she went in witho "
 toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe 

In [38]:
model.fit(X, y, epochs = 200, batch_size = 2048)

Epoch 1/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.3553
Epoch 2/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.3424
Epoch 3/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.3298
Epoch 4/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.3154
Epoch 5/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.3060
Epoch 6/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.2951
Epoch 7/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.2818
Epoch 8/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.2706
Epoch 9/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.2602
Epoch 10/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.2481
Epoch 11/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.23

163717/163717 [==============================] - 28s 169us/step - loss: 1.7950
Epoch 91/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.7908
Epoch 92/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.7869
Epoch 93/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.7771
Epoch 94/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.7816
Epoch 95/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.7801
Epoch 96/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.7811
Epoch 97/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.7751
Epoch 98/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.7737
Epoch 99/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.7653
Epoch 100/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.7591


163717/163717 [==============================] - 28s 169us/step - loss: 1.6301
Epoch 179/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.6185
Epoch 180/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.6165
Epoch 181/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.6161
Epoch 182/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.6200
Epoch 183/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.6233
Epoch 184/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.6293
Epoch 185/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.6209
Epoch 186/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.6174
Epoch 187/200
163717/163717 [==============================] - 28s 171us/step - loss: 1.6110
Epoch 188/200
163717/163717 [==============================] - 28s 169us/step - loss

In [39]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print("\nDone.")

Seed:
" -frame, or something of the sort.

next came an angry voice--the rabbit’s--‘pat! pat! where are you? "
’

‘he ding wuitei sue of the cane  sh ceow! io wou dol lede toer ’iu fanet to te the haoter tn the tei. aud shrt wi thal sht wast teen s shen hame a arin and a fatee sa soahe wottg to moke ’hu go toutend io the sane thi  and wein is totl an an aace aod, tiad a lorgle hearee, and sheu hne titt aliie whsh sn hnrsel. ‘ih wou doo the moot of the gint an th she whit h tait to the koreronni to the hamde.

‘hel a din hor here ay all ali ailiti an atl aelit th tee teessd ’ aad a donwlrnsel ioputent on oone then  ane to do wout the whet  in wound fo tae iott of the gant, aad a lirtle thich a lailee at an  ald e gon  fo a loade to thi the that an the tirtoe of the saad thtt bf tne cal an the calle. ‘ou meas t she said to hheself, at sae had no ven ergluiin tein ites to then at the sagted het  and theughe th the tart tfeisi gor the rioe the was toteli then woued an toel i sheee thttle t

In [40]:
model.fit(X, y, epochs = 200, batch_size = 2048)

Epoch 1/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5808
Epoch 2/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5841
Epoch 3/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5776
Epoch 4/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5866
Epoch 5/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5859
Epoch 6/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5757
Epoch 7/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5792
Epoch 8/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5695
Epoch 9/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5776
Epoch 10/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5687
Epoch 11/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.56

163717/163717 [==============================] - 28s 168us/step - loss: 2.6630
Epoch 94/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.6483
Epoch 95/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.6336
Epoch 96/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.6163
Epoch 97/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.6006
Epoch 98/200
163717/163717 [==============================] - 28s 168us/step - loss: 2.5838
Epoch 99/200
163717/163717 [==============================] - 28s 168us/step - loss: 2.5686
Epoch 100/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.5521
Epoch 101/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.5361
Epoch 102/200
163717/163717 [==============================] - 28s 169us/step - loss: 2.5198
Epoch 103/200
163717/163717 [==============================] - 28s 168us/step - loss: 2.50

163717/163717 [==============================] - 28s 169us/step - loss: 1.5737
Epoch 182/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5695
Epoch 183/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5763
Epoch 184/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5781
Epoch 185/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5624
Epoch 186/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5617
Epoch 187/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5560
Epoch 188/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5441
Epoch 189/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5494
Epoch 190/200
163717/163717 [==============================] - 28s 169us/step - loss: 1.5389
Epoch 191/200
163717/163717 [==============================] - 28s 169us/step - loss

In [41]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print("\nDone.")

Seed:
" holding her hand on the top of her head to feel which way it was
growing, and she was quite surprise "
d to tei dttliused th the kihte th hea feae with soeye 
ant saed in tistled tn the ooher.

‘h sast t said the moek turtle: ‘woul the wer 
so doan an ay s
shi shid uo the ali totie tnterfeng to aerin in one.

‘if porene thing in the docw tae the gurarss tniee-’ she said to herself, sto wou doont anf dork thet here an serering and then it tootle oh be inle wo then, thte an the was a little oo st the wastin an oo eo the dareepiller.

‘hede if ane wout an “our oay,’ taid alice in a sera of thict hrdee.
and shen see werter an toe want te thet sere the dande hat and to deve peane whst saresg anonss, and test iu had io the buihesss ceore thined. and tone i cei’t han  nouc co whet tound toung to bete in the sonte oi the hand  nhe more toitting oo hanted. 
‘when if yhu gren toe gan,’ taed alice, ‘ard tou dou’t sale tou dn wou th thin, i whntt is woul oo biong an mn eo ths oose ’uur lak